In [2]:
import numpy as np
import math
import pandas as pd

The dispersion Parameter in revosim works that the smaller it is the bigger it is the displacement. In this code I'm first trying to calculate the max px displacement for a given set of sispersion paramenter and then I'll try to check what dispersion parameters I need to have a certain amount of displacement.

In [ ]:
print(list(range(1, 15, 1)))

In [22]:
#d = [2, 5, 10, 15, 20, 30, 40, 50, 60, 70]
d = list(range(15))
b = list(range(256))  # All integers from 0 to 255
a = list(range(361))  # All integers from 0 to 360

#x = ( math.sqrt( ( 65536 ) / ( b + 1 ) ) - 16 ) * ( ( np.sin(a) ) / ( d ))


In [23]:
# empty list to store result of x
results = []

# loop over each combination of d, b, and a
for di in d:
    for bi in b:
        for ai in a:
            # find the value of x
            x = (math.sqrt((65536) / (bi + 1)) - 16) * (np.sin(ai) / di)
            
            # Append the result to the list
            results.append([di, bi, ai, x])

# Create a DataFrame to store the results
df = pd.DataFrame(results, columns=['d', 'b', 'a', 'x'])

/var/folders/dm/ryjvp3j51h78gsbgf92k6xvh0000gp/T/ipykernel_49805/4240695442.py:9: RuntimeWarning: invalid value encountered in scalar divide
  x = (math.sqrt((65536) / (bi + 1)) - 16) * (np.sin(ai) / di)
/var/folders/dm/ryjvp3j51h78gsbgf92k6xvh0000gp/T/ipykernel_49805/4240695442.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  x = (math.sqrt((65536) / (bi + 1)) - 16) * (np.sin(ai) / di)
/var/folders/dm/ryjvp3j51h78gsbgf92k6xvh0000gp/T/ipykernel_49805/4240695442.py:9: RuntimeWarning: invalid value encountered in scalar multiply
  x = (math.sqrt((65536) / (bi + 1)) - 16) * (np.sin(ai) / di)


In [24]:
# Group by 'd' and get the indices of max and min values of 'x'
max_x = df.groupby('d')['x'].idxmax()
min_x = df.groupby('d')['x'].idxmin()

# Get the rows corresponding to the max and min values of 'x' for each 'd'
max_x_rows = df.loc[max_x]
min_x_rows = df.loc[min_x]

# Add a column to identify if the row contains the max or min value
max_x_rows['type'] = 'max'
min_x_rows['type'] = 'min'

# Concatenate the DataFrames
max_min_x_rows = pd.concat([max_x_rows, min_x_rows])

In [25]:
# Save the max and min values to a single CSV file
max_min_x_rows.to_csv('dispersal-ranges.csv', index=False)